In [9]:
from __future__ import print_function
import pickle
import pandas as pd
import numpy as np
import config
import matplotlib.pyplot as plt
%matplotlib inline

# more info in https://github.com/marcotcr/lime 
import lime
import lime.lime_tabular
from lime.lime_tabular import LimeTabularExplainer
np.random.seed(1)

In [4]:
def load_model(modelname):

    # load the model from disk
    loaded_model = pickle.load(open(f"{config.MODEL_PATH}{modelname}.bin", 'rb'))

    return loaded_model

    pass

In [5]:
def load_data():

    # read the training data
    X_train =  pickle.load(open( f'{config.INPUT_PATH}X_train.pkl', "rb" )) 
    y_train = pickle.load(open( f'{config.INPUT_PATH}y_train.pkl', "rb" ))

    # read test data
    X_test = pickle.load(open( f'{config.INPUT_PATH}X_test.pkl', "rb" )) 
    y_test = pickle.load(open( f'{config.INPUT_PATH}y_test.pkl', "rb" )) 

    return X_train, y_train, X_test, y_test


### Load Datasets

In [7]:
# load train and validation datasets
X_train, y_train, X_test, y_test = load_data() 

#### Create the explainer
As opposed to lime_text.TextExplainer, tabular explainers need a training set. 
The reason for this is because we compute statistics on each feature (column). 
If the feature is `numerical`, we compute the mean and std, and discretize it into quartiles. 
If the feature is `categorical`, we compute the frequency of each value. 


The parameters passed to the explainer are:
- our training set, we need to make sure we use the training set *without* one hot encoding
- `mode`: the explainer can be used for classification or regression
- `feature_names`: list of labels for our features
- `categorical_features`: list of indexes of categorical features
- `categorical_names`: dict mapping each index of categorical feature to a list of corresponding labels
- `dicretize_continuous`: will discretize numerical values into buckets that can be used for explanation. For instance it can tell us that the decision was made because distance is in bucket [5km, 10km] instead of telling us distance is an importante feature.

In [17]:
X_train.columns.tolist()

['PRODUCTO_11_saldo__rolling_std_w6',
 'PRODUCTO_1_saldo__diff_p3',
 'PRODUCTO_21_saldo__rolling_mean_w3',
 'PRODUCTO_12_saldo__rolling_sum_t6',
 'PRODUCTO_27_COD_CLASIFICACION_DEUDOR_',
 'PRODUCTO_10_saldo__rolling_std_w6',
 'PRODUCTO_13_saldo_',
 'PRODUCTO_33_tipo_credito_',
 'PRODUCTO_14_saldo__pct_p1',
 'PRODUCTO_13_saldo__rolling_sum_t6',
 'PRODUCTO_36_saldo__diff_p1',
 'PRODUCTO_3_saldo__diff_p3',
 'PRODUCTO_41_saldo__rolling_mean_w3',
 'PRODUCTO_7_saldo__pct_p6',
 'PRODUCTO_29_saldo__pct_p3',
 'PRODUCTO_1_condicion_',
 'PRODUCTO_34_condicion_',
 'PRODUCTO_16_saldo__rolling_mean_w3',
 'PRODUCTO_31_saldo__diff_p3',
 'PRODUCTO_15_RIESGO_DIRECTO_',
 'PRODUCTO_0_saldo__count',
 'PRODUCTO_30_RIESGO_DIRECTO_',
 'PRODUCTO_39_RIESGO_DIRECTO_',
 'PRODUCTO_29_saldo__count',
 'PRODUCTO_35_saldo__pct_p1',
 'PRODUCTO_4_saldo__pct_p6',
 'PRODUCTO_25_saldo__rolling_mean_w6',
 'PRODUCTO_23_condicion_',
 'PRODUCTO_9_saldo__rolling_std_w6',
 'PRODUCTO_6_saldo__rolling_sum_t6',
 'PRODUCTO_27_saldo_

In [20]:
X_train.loc[:,['sexo',
 'dto', 'PRODUCTO_7_condicion_', 'PRODUCTO_7_condicion_']]

,sexo,dto,PRODUCTO_7_condicion_,PRODUCTO_7_condicion_
key_value,,,,
200315,0.0,100.0,-9999.0,-9999.0
34712,1.0,343.0,0.0,0.0
67855,0.0,1313.0,-9999.0,-9999.0
144799,0.0,134.0,-9999.0,-9999.0
93696,1.0,478.0,-9999.0,-9999.0
...,...,...,...,...
136751,0.0,100.0,-9999.0,-9999.0
332194,1.0,1269.0,0.0,0.0
17513,1.0,198.0,-9999.0,-9999.0


In [16]:
X_train.nunique().sort_values(ascending=True).reset_index().head(20)

,index,0
0,PRODUCTO_36_saldo__diff_p1,1
1,PRODUCTO_20_condicion_,2
2,PRODUCTO_35_saldo__pct_p1,2
3,PRODUCTO_39_RIESGO_DIRECTO_,2
4,PRODUCTO_33_saldo__diff_p1,2
5,PRODUCTO_27_COD_CLASIFICACION_DEUDOR_,2
6,PRODUCTO_30_RIESGO_DIRECTO_,2
7,PRODUCTO_15_RIESGO_DIRECTO_,2
8,PRODUCTO_25_RIESGO_DIRECTO_,2
9,PRODUCTO_41_saldo__rolling_mean_w3,2


In [ ]:
categorical_names = {}
for col in cat_features:
    categorical_names[X_train.columns.get_loc(col)] = [new_col.split("__")[1] 
                                                       for new_col in new_ohe_features 
                                                       if new_col.split("__")[0] == col]

In [8]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=X_train.columns, class_names='target', discretize_continuous=True)

TypeError: '(slice(None, None, None), 0)' is an invalid key

In [ ]:
X_train.